[View in Colaboratory](https://colab.research.google.com/github/linked0/deep-learning/blob/master/Batch_Normalization_Lesson_181015.ipynb)

In [1]:
!pip install tqdm

In [3]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [4]:
# Import necessary packages
import tensorflow as tf
import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import MNIST data so we have something for out experiments
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
class NerualNet:
  def __init__(self, initial_weights, activation_fn, use_batch_norm):
    # Keep track of whether or not this network uses batch normalization.
    self.use_batch_norm = use_batch_norm
    self.name = 'With Batch Norm' if use_batch_norm else 'Without Batch Norm'
    
    # Batch normalization needs to do different calculations during training and inference,
    # so we use this placeholder to tell the graph which behavior to use.
    self.is_training = tf.placeholder(tf.bool, name='is_training')
    
    self.training_accuracies = []
    
    self.build_network(initial_weights, activation_fn)
    
  def build_network(self, initial_weights, activation_fn):
    self.input_layer = tf.placeholder(tf.flaot32, [None, initial_weights[0].shape[0]])
    layer_in = self.input_layer
    for weights in initial_weights[:-1]:
      layer_in = self.fully_connected(layer_in, weights, activation_fn)
    self.output_layer = self.fully_connected(layer_in, initial_weights[-1])
  
  def fully_connected(self, layer_in, initial_weights, activation_fn=None):
    if self.use_batch_norm and activation_fn:
      weights = tf.Variable(initial_weights)
      
      batch_normalized_output = tf.layers.batch_normalization(linear_output, training=self.is_training)
      
      return activatiion_fn(batch_normalized_output)
    
    else:
      weights = tf.Variable(initial_weights)
      biases = tf.Variable(tf.zeros([initial_weights.shape[-1]]))
      linear_output = tf.add(tf.matmul(layer_in, weights), biases)
      return linear_output if not activation_fn else activation_fn(linear_output)
    
    